# Приветствую !

Итак, что же делает эта непонятная х***ь. Если вы постоянно пробуете кучу разных экспериментов, используя DeepPavlov. И для того чтобы провести новый эксперимент вам каждый раз треубется залезать в конфиг, убирать что-то из chainer-а, ручками переписывать параметры, потом снова всё это запускать, а вам всё это делать некогда, так как вы очень занятой человек (скорее всего это просто лень), и хочется просто запустить в консоли одну волшебную команду, то вы мой клиент, и эта штука точно облегчит вам в жизнь (на самом деле не точно). 

# Введение

Вот вам конкретный пример. Решается задача классивикации, допустим интентов. У вас есть 10 моделей (которые могут её решать), несколько токенайзеров, опечаточник, лемматизатор, ELMo, fasttext, и много чего ещё. И вы хотите автоматически прогнать кучу экспериментов с разными моделями и компонентами, чтобы в итоге посмотреть как эти модели себя покажут себя на практике, что круче fasstext или ELMo, даёт ли присутствие лемматизации с опечаточником в пайплайне какой-нибудь буст, и желательно всё сразу. И неплохо было бы ещё при этом гиперапараметры моделей попереберать. Ну в общем pipeline_manager как раз и делает этот самый автоматический перебор различных компонент, создаёт из их комбинаций разные пайплайны, и запускает их, после чего формирует некую статистику на основании логов.

Но сначала вам требуется явно задать шаблон перебора разных компонент. Как все мы знаем при написании конфига для эксперимента мы в поле "chainer" задаём "pipe" в виде списка словарей, которые описывают последовательность компонент и их параметров. Вот пример:

In [ ]:
ch_example = {"chainer": {"in": ["x"], "in_y": ["y"],
                          "pipe": [
                                    {"id": "classes_vocab",
                                     "name": "default_vocab",
                                     "fit_on": ["y"],
                                     "level": "token",
                                     "save_path": "vocabs/snips_classes.dict",
                                     "load_path": "vocabs/snips_classes.dict"},
                                    {"id": "my_embedder",
                                     "name": "fasttext",
                                     "save_path": "embeddings/dstc2_fastText_model.bin",
                                     "load_path": "embeddings/dstc2_fastText_model.bin",
                                     "dim": 100},
                                    {"in": ["x"],
                                     "name": "str_lower",
                                     "out": ["x"]},
                                    {
                                      "id": "my_tokenizer",
                                      "name": "nltk_tokenizer",
                                      "tokenizer": "wordpunct_tokenize"
                                    },
                                    {
                                    "in": ["x"],
                                    "in_y": ["y"],
                                    "out": ["y_labels", "y_probas_dict"],
                                    "main": True,
                                    "scratch_init": True,
                                    "name": "intent_model",
                                    "save_path": "intents/intent_cnn_snips_v4",
                                    "classes": "#classes_vocab.keys()",
                                    "kernel_sizes_cnn": [1, 2, 3],
                                    "filters_cnn": 256,
                                    "confident_threshold": 0.5,
                                    "optimizer": "Adam",
                                    "lear_rate": 0.01,
                                    "lear_rate_decay": 0.1,
                                    "loss": "binary_crossentropy",
                                    "text_size": 15,
                                    "coef_reg_cnn": 1e-4,
                                    "coef_reg_den": 1e-4,
                                    "dropout_rate": 0.5,
                                    "dense_size": 100,
                                    "model_name": "cnn_model",
                                    "embedder": "#my_embedder",
                                    "tokenizer": "#my_tokenizer"},
                                  ],
                          "out": ["y_labels", "y_probas_dict"]}}

В данной случае на каждом этапе исполнения пайплайна присутствет лишь одна компонента. В общем-то и перебирать нечего, так что давайте кое что добавим:

In [ ]:
{"chainer": {"in": ["x"], "in_y": ["y"],
    "pipe": [
      [{"id": "classes_vocab",
        "name": "default_vocab",
        "fit_on": ["y"],
        "level": "token",
        "save_path": "vocabs/snips_classes.dict",
        "load_path": "vocabs/snips_classes.dict"}],
      [{"id": "my_embedder",
        "name": "fasttext",
        "save_path": "embeddings/dstc2_fastText_model.bin",
        "load_path": "embeddings/dstc2_fastText_model.bin",
        "dim": 100}],
      [{"in": ["x"],
        "name": "str_lower",
        "out": ["x"]},
        None],
      [{"id": "my_tokenizer",
        "name": "nltk_tokenizer",
        "tokenizer": "wordpunct_tokenize"},
       {"id": "my_tokenizer",
        "name": "lazy_tokenizer",
        "tokenizer": "wordpunct_tokenize"}],
      [{"in": ["x"], "in_y": ["y"],
        "out": ["y_labels", "y_probas_dict"],
        "main": True,
        "scratch_init": True,
        "name": "intent_model",
        "save_path": "intents/intent_cnn_snips_v4",
        "classes": "#classes_vocab.keys()",
        "kernel_sizes_cnn": [1, 2, 3],
        "filters_cnn": 256,
        "confident_threshold": 0.5,
        "optimizer": "Adam",
        "lear_rate": 0.01,
        "lear_rate_decay": 0.1,
        "loss": "binary_crossentropy",
        "text_size": 15,
        "coef_reg_cnn": 1e-4,
        "coef_reg_den": 1e-4,
        "dropout_rate": 0.5,
        "dense_size": 100,
        "model_name": "cnn_model",
        "embedder": "#my_embedder",
        "tokenizer": "#my_tokenizer"},
       {"in": ["x"], "in_y": ["y"],
        "out": ["y_labels", "y_probas_dict"],
        "main": True,
        "scratch_init": True,
        "name": "intent_model",
        "save_path": "intents/intent_cnn_snips_v4",
        "classes": "#classes_vocab.keys()",
        "units_lstm": 128,
        "rec_dropout_rate": 0.5,
        "confident_threshold": 0.5,
        "optimizer": "Adam",
        "lear_rate": 0.01,
        "lear_rate_decay": 0.1,
        "loss": "binary_crossentropy",
        "text_size": 15,
        "coef_reg_lstm": 1e-4,
        "coef_reg_den": 1e-4,
        "dropout_rate": 0.5,
        "dense_size": 100,
        "model_name": "bilstm_model",
        "embedder": "#my_embedder",
        "tokenizer": "#my_tokenizer"}]
       ],
    "out": ["y_labels", "y_probas_dict"]}
}

Основное отличие в том что здесь каждый элемент списка в поле "pipe", сам является списком, даже если он содержит один компонент. Ну и как можно видеть теперь у нас в конфиге два токенайзера (на самом деле они одинаковые, но в DeepPavlov это разные классы), и две модели CNN и bi-LSTM. Также обратите внимание что третий элемент кроме словаря также содержит None. Это значит, что этот элемент может и вовсе отсутствовать в пайплайне, и будут созданы запущены пайплайны не содержащие этого компонента.

Вот в общем-то и всё. Для того чтобы задать шаблон перебора вам требуется взять ваш конфиг, который вы уже использовали до этого и дописать в "pipe" chainer-a дополнительные компоненты и модели, которые вы хотите попробовать. Главное не забыть что каждый элемент в новом "pipe" является отдельным списком. После чего уже можно запустить эксперимент через командную строку, введя:

python -m deeppavlov sort_out <путь к новому конфиг файлу> -e <имя эксперимента>

После чего процесс запустится. По окончании в папке Downloads появится папка Experiments, в которой будут сохраняться все ваши данные, отчёты, чекпоинты по отдельным экспериментам рассортированные по датам и именам экспериментов.

In [ ]:
from deeppavlov.pipeline_manager.pipeline_manager import PipelineManager
from deeppavlov.core.commands.utils import set_deeppavlov_root, expand_path
from deeppavlov.download import deep_download

set_deeppavlov_root({})
data_path = expand_path('snips')

path = '/home/mks/projects/DeepPavlov/deeppavlov/configs/my_configs/intents/intents_snips.json'
exp_name = 'test_10'
mode = 'train'
root = '/home/mks/projects/DeepPavlov/experiments/'
hyper_search = 'grid'
sample_num = 10
target_metric = 'classification_f1'


def main():

    deep_download(['-c', path])

    manager = PipelineManager(config_path=path, exp_name=exp_name, mode=mode, root=root,
                              hyper_search=hyper_search, sample_num=sample_num, target_metric=target_metric)
    manager.run()


if __name__ == "__main__":
    main()